# Process GWTC data

This notebook uses the PE samples from the GWTC-2.1 and GWTC-3 data release for use in model selection. p_theta_jcb is calculated analytically as in Cheng et al. 2023, while using a different d_L interpolation than Cheng et al. 2023 to ensure there are no p_theta=0 samples in the data. Last updated 21-10-2024, Modified from April's Process GWTC data NEW notebook

In [2]:
import h5py
import numpy as np
import pandas as pd
import os
import re

import matplotlib.pyplot as plt

from astropy import cosmology
from astropy.cosmology import z_at_value
import astropy.units as u
cosmo = cosmology.Planck15

from scipy.interpolate import UnivariateSpline
from scipy.stats import gaussian_kde
from tqdm import tqdm
import bilby

sys.path.append('../../AMAZE_project_resources/')
from effectivespinpriors.priors import chi_effective_prior_from_isotropic_spins


In [3]:
output_directory = '/data/wiay/2297403c/amaze_model_select/Nflows_AMAZE_paper/inputs/GWTC-3/events'
removed_events_directory = '/data/wiay/2297403c/amaze_model_select/Nflows_AMAZE_paper/inputs/GWTC-3/removed'

#where PE files are stored
GWTC21_path = '/data/wiay/pe_samples_releases/gwtc2.1'
GWTC3_path = '/data/wiay/pe_samples_releases/gwtc3'

#all GWs with FAR < 1/yr
gw_names_path = '/data/wiay/2297403c/amaze_model_select/Nflows_AMAZE_paper/inputs/GWTC-3/gw_names.txt'
with open(gw_names_path, 'r') as f:
    gw_names = [line.strip() for line in f.readlines()]
print(gw_names)

['GW150914_095045', 'GW151012_095443', 'GW151226_033853', 'GW170104_101158', 'GW170608_020116', 'GW170729_185629', 'GW170809_082821', 'GW170814_103043', 'GW170818_022509', 'GW170823_131358', 'GW190408_181802', 'GW190412_053044', 'GW190413_134308', 'GW190421_213856', 'GW190503_185404', 'GW190512_180714', 'GW190513_205428', 'GW190517_055101', 'GW190519_153544', 'GW190521_030229', 'GW190521_074359', 'GW190527_092055', 'GW190602_175927', 'GW190620_030421', 'GW190630_185205', 'GW190701_203306', 'GW190706_222641', 'GW190707_093326', 'GW190708_232457', 'GW190720_000836', 'GW190727_060333', 'GW190728_064510', 'GW190803_022701', 'GW190828_063405', 'GW190828_065509', 'GW190910_112807', 'GW190915_235702', 'GW190924_021846', 'GW190925_232845', 'GW190929_012149', 'GW190930_133541', 'GW191105_143521', 'GW191109_010717', 'GW191127_050227', 'GW191129_134029', 'GW191204_171526', 'GW191215_223052', 'GW191216_213338', 'GW191222_033537', 'GW191230_180458', 'GW200112_155838', 'GW200128_022011', 'GW200129_0

In [4]:
# create interpolant for determining redshifts from luminosity distances
dL_max = 20000 # Mpc
dL_vals = np.logspace(-4, np.log10(dL_max), 10000)
redshift_vals = np.asarray([z_at_value(cosmo.luminosity_distance, dL*u.Mpc) for dL in dL_vals])
z_from_dL = UnivariateSpline(dL_vals, redshift_vals)
dzdL_from_dL = z_from_dL.derivative()

def dL_to_redshift(dL):
    # Input distance in Mpc
    # Use interpolant above to speed this up
    return z_from_dL(dL)

In [5]:
dL_to_redshift(80.)

array(0.01693232)

In [6]:
# conversion function

def m1m2_to_mchirp(m1,m2):
    return (m1*m2)**(3./5) / (m1+m2)**(1./5)

def m1m2_to_mtot(m1,m2):
    return m1+m2

def m1m2_to_q(m1,m2):
    # q is defined as m2/m1 with m2<=m1
    q = m2/m1
    pos_idxs = np.argwhere(q > 1)
    q[pos_idxs] = m1[pos_idxs]/m2[pos_idxs]
    return q

def m1m2_to_eta(m1,m2):
    eta = (m1*m2) / (m1+m2)**2
    return eta

def components_to_chieff(m1,m2,a1,a2,costilt1,costilt2):
    return (m1*a1*costilt1 + m2*a2*costilt2) / (m1+m2)

def prob(priors, sample): #why is this not predefined in bilby lmfao
    return np.prod(np.array([priors[key].prob(sample[key]) for key in sample]), axis = 0)

In [7]:
np.random.seed(12) #set for reproducability

## GWTC-2.1

In [13]:
no_prior = [] #some samples have no analytic prior given; do these later separately
prior_params = ['mass_1', 'mass_2', 'chirp_mass', 'mass_ratio', 'luminosity_distance']
prior_params_z = ['mass_1', 'mass_2', 'chirp_mass', 'mass_ratio', 'redshift']

for path in (GWTC21_path, GWTC3_path):
    
    os.chdir(path)

    for filename in sorted(os.listdir(path)):

        gw_name = filename.split('-')[-1].split('_PE')[0]
        if gw_name+'.hdf5' in os.listdir(output_directory):
            print(f'skipping {gw_name} - done')
            continue

        #use uniform comoving volume prior
        if 'nocosmo' in filename or 'prior' in filename or '.h5' not in filename:
            continue 

        if gw_name in gw_names: #only use confident BBHs

            print(gw_name)

            with h5py.File(filename, 'r') as f:

                #some events are missing priors (data release error)
                if 'analytic' not in f['C01:IMRPhenomXPHM']['priors'].keys(): 
                    no_prior.append(gw_name)
                    print(f'skipping {gw_name} - no prior')
                    continue

                #read in PE samples
                df = pd.DataFrame(np.asarray(f['C01:Mixed']['posterior_samples'])) 

                # luminosity distance and redshift
                dL = np.asarray(df['luminosity_distance'])
                z = np.asarray(df['redshift'])

                # chirp mass and total mass (source frame)
                m1_src = np.asarray(df['mass_1_source'])
                m2_src = np.asarray(df['mass_2_source'])
                mchirp = np.asarray(df['chirp_mass_source'])
                mtot = np.asarray(df['total_mass_source'])

                # q and eta
                q = np.asarray(df['mass_ratio'])
                eta = np.asarray(df['symmetric_mass_ratio'])

                # chieff
                chieff = components_to_chieff(m1_src, m2_src, np.asarray(df['a_1']), np.asarray(df['a_2']), \
                                              np.asarray(df['cos_tilt_1']), np.asarray(df['cos_tilt_2']))

                new_df = pd.DataFrame({'m1':m1_src, 'm2':m2_src, 'mchirp':mchirp, 'mtot':mtot, 'q':q, 'eta':eta, 
                                       'chieff':chieff, 'dL':dL, 'z':z})         

                # calculate prior at posterior points
                with open(f'{output_directory}/../priors/' + gw_name+'_prior.prior', 'w') as prior_f:
                    a_max = float(f['C01:IMRPhenomXPHM']['priors']['analytic']['a_1'][0].\
                                  decode('UTF8').split('maximum=')[1].split(',')[0])
                    for param in prior_params:
                        if param in ['chirp_mass', 'mass_ratio']:
                            prior_f.write(param + ' = bilby.gw.prior.' + 
                                          f['C01:IMRPhenomXPHM']['priors']['analytic'][param][0].decode('UTF8') + '\n')
                        else:
                            prior_f.write(param + ' = ' + 
                                          f['C01:IMRPhenomXPHM']['priors']['analytic'][param][0].decode('UTF8') + '\n')

            priors = bilby.core.prior.PriorDict(filename=f'{output_directory}/../priors/' + gw_name+'_prior.prior')

            if priors['luminosity_distance'].minimum > np.min(dL)
                ld_min = np.min(dL)
            else:
                ld_min = priors['luminosity_distance'].minimum
            ld_max = priors['luminosity_distance'].maximum
            z_min = dL_to_redshift(ld_min)
            z_max = dL_to_redshift(ld_max)

            priors.pop('luminosity_distance')
            priors['redshift'] = bilby.gw.prior.UniformSourceFrame(z_min, z_max, name='redshift')

            p = prob(priors, df[prior_params_z]) #mchirp_det, q, z

            p *= (1 + new_df['z']) #mchirp_det to mchirp_source jacobian

            # apply chieff jacobian from tcallister's github
            p_chieffs = []
            for q_i, chieff_i in zip(new_df['q'], new_df['chieff']):
                p_chieffs.append(chi_effective_prior_from_isotropic_spins(q_i, a_max, chieff_i)[0]) #p(chieff | q)
            p *= np.array(p_chieffs)

            new_df['p_theta_jcb'] = p

            new_df.to_hdf(os.path.join(output_directory,gw_name+'.hdf5'), key='combined')

# remove GW190521
os.rename(os.path.join(output_directory, 'GW190521_030229.hdf5'), os.path.join(removed_events_directory, 'GW190521_030229.hdf5'))

skipping GW150914_095045 - done
skipping GW150914_095045 - done
skipping GW150914_095045 - done
skipping GW151012_095443 - done
skipping GW151012_095443 - done
skipping GW151012_095443 - done
skipping GW151226_033853 - done
skipping GW151226_033853 - done
skipping GW151226_033853 - done
skipping GW170104_101158 - done
skipping GW170104_101158 - done
skipping GW170104_101158 - done
GW170608_020116
skipping GW170608_020116 - no prior
skipping GW170729_185629 - done
skipping GW170729_185629 - done
skipping GW170729_185629 - done
skipping GW170809_082821 - done
skipping GW170809_082821 - done
skipping GW170809_082821 - done
skipping GW170814_103043 - done
skipping GW170814_103043 - done
skipping GW170814_103043 - done
skipping GW170818_022509 - done
skipping GW170818_022509 - done
skipping GW170818_022509 - done
skipping GW170823_131358 - done
skipping GW170823_131358 - done
skipping GW170823_131358 - done
skipping GW190408_181802 - done
skipping GW190408_181802 - done
skipping GW190408_18

/tmp/ipykernel_3735404/4216783787.py:78: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  p = prob(priors, df[prior_params_z]) #mchirp_det, q, z


skipping GW190521_030229 - done
skipping GW190521_030229 - done
skipping GW190521_074359 - done
skipping GW190521_074359 - done
skipping GW190521_074359 - done
skipping GW190527_092055 - done
skipping GW190527_092055 - done
skipping GW190527_092055 - done
skipping GW190602_175927 - done
skipping GW190602_175927 - done
skipping GW190602_175927 - done
skipping GW190620_030421 - done
skipping GW190620_030421 - done
skipping GW190620_030421 - done
skipping GW190630_185205 - done
skipping GW190630_185205 - done
skipping GW190630_185205 - done
skipping GW190701_203306 - done
skipping GW190701_203306 - done
skipping GW190701_203306 - done
skipping GW190706_222641 - done
skipping GW190706_222641 - done
skipping GW190706_222641 - done
GW190707_093326
skipping GW190707_093326 - no prior
skipping GW190708_232457 - done
skipping GW190708_232457 - done
skipping GW190708_232457 - done
skipping GW190719_215514 - done
skipping GW190719_215514 - done
skipping GW190719_215514 - done
GW190720_000836
skip

/tmp/ipykernel_3735404/4216783787.py:78: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  p = prob(priors, df[prior_params_z]) #mchirp_det, q, z


skipping GW191216_213338 - done
skipping GW191216_213338 - done
skipping GW191222_033537 - done
skipping GW191222_033537 - done
skipping GW191222_033537 - done
skipping GW191222_033537 - done
skipping GW191222_033537 - done
skipping GW191222_033537 - done
skipping GW191230_180458 - done
skipping GW191230_180458 - done
skipping GW191230_180458 - done
skipping GW191230_180458 - done
skipping GW191230_180458 - done
skipping GW191230_180458 - done
skipping GW200112_155838 - done
skipping GW200112_155838 - done
skipping GW200112_155838 - done
skipping GW200112_155838 - done
skipping GW200112_155838 - done
skipping GW200112_155838 - done
skipping GW200128_022011 - done
skipping GW200128_022011 - done
skipping GW200128_022011 - done
skipping GW200128_022011 - done
skipping GW200128_022011 - done
skipping GW200128_022011 - done
skipping GW200129_065458 - done
skipping GW200129_065458 - done
skipping GW200129_065458 - done
skipping GW200129_065458 - done
skipping GW200129_065458 - done
skipping

/tmp/ipykernel_3735404/4216783787.py:78: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  p = prob(priors, df[prior_params_z]) #mchirp_det, q, z


skipping GW200225_060421 - done
skipping GW200225_060421 - done
skipping GW200302_015811 - done
skipping GW200302_015811 - done
skipping GW200302_015811 - done
skipping GW200302_015811 - done
skipping GW200302_015811 - done
skipping GW200302_015811 - done
skipping GW200311_115853 - done
skipping GW200311_115853 - done
skipping GW200311_115853 - done
skipping GW200311_115853 - done
skipping GW200311_115853 - done
skipping GW200311_115853 - done
skipping GW200316_215756 - done
skipping GW200316_215756 - done
skipping GW200316_215756 - done
skipping GW200316_215756 - done
skipping GW200316_215756 - done
skipping GW200316_215756 - done


In [11]:
no_prior

['GW170608_020116',
 'GW190707_093326',
 'GW190720_000836',
 'GW190725_174728',
 'GW190728_064510',
 'GW190924_021846']

In [10]:
import glob as glob
gw_path="/data/wiay/2297403c/amaze_model_select/Nflows_AMAZE_paper/inputs/GWTC-3/events"
params = ['mchirp', 'q', 'chieff', 'z']

def plot_0prior_samps(gw_path):
    for filename in sorted(os.listdir(gw_path)):
        gw = pd.read_hdf(gw_path+ '/'+filename)
        prior = gw['p_theta_jcb']

        if np.any(prior==0.):
            fig,ax=plt.subplots(1,4)
            for i in range(4):
                ax[i].scatter(np.array(gw[params[i]])[prior==0.], 0.2*np.ones(len(np.array(gw[params[i]])[prior==0.])), s=1.)
                ax[i].hist(gw[params[i]], bins=40,alpha=0.2, density=True)
            print(filename)
            print(np.min(prior))
            print(np.array(gw['z'])[prior==0.])
            print(np.array(gw['dL'])[prior==0.])
        #plt.hist(np.log(prior), bins=40, range=(-30,0), alpha=0.5)

def plot_0prior_samps_2d(gw_path, params):
    for filename in sorted(os.listdir(gw_path)):
        gw = pd.read_hdf(gw_path+ '/'+filename)
        prior = gw['p_theta_jcb']

        if np.any(prior==0.):
            fig,ax=plt.subplots(1,1)
            ax.scatter(np.array(gw[params[0]])[prior==0.], np.array(gw[params[1]])[prior==0.], s=1.)
            ax.hist2d(gw[params[0]], gw[params[1]], bins=40,alpha=0.5, density=True)
            print(filename)
        #plt.hist(np.log(prior), bins=40, range=(-30,0), alpha=0.5)

In [14]:
plot_0prior_samps('/data/wiay/2297403c/amaze_model_select/Nflows_AMAZE_paper/inputs/GWTC-3/events')

IsADirectoryError: ``/data/wiay/2297403c/amaze_model_select/Nflows_AMAZE_paper/inputs/GWTC-3/events/priors`` is not a regular file

## Events that are missing analytical prior

I ended up getting the priors from each event's repo:

- GW170608_020116 - https://git.ligo.org/pe/O2/GW170608/-/blob/master/C01_offline/ProdF13.prior 
- GW190707_093326 - https://git.ligo.org/pe/O3/S190707q/-/blob/master/C01_offline/ProdF9.prior 
- GW190720_000836 - https://git.ligo.org/pe/O3/S190720a/-/blob/master/C01_offline/ProdF11.prior
- GW190725_174728 - https://git.ligo.org/pe/O3/S190725t/-/blob/master/C01_offline/ProdF15.prior
- GW190728_064510 - https://git.ligo.org/pe/O3/S190728q/-/blob/master/C01_offline/ProdF5.prior
- GW190924_021846 - https://git.ligo.org/pe/O3/S190924h/-/blob/master/C01_offline/ProdF7.prior

In [1]:
no_prior = ['GW170608_020116',
 'GW190707_093326',
 'GW190720_000836',
 'GW190725_174728',
 'GW190728_064510',
 'GW190924_021846']

In [9]:
os.chdir(GWTC21_path)

In [15]:
prior_params_z = ['mass_1', 'mass_2', 'chirp_mass', 'mass_ratio', 'redshift']

for gw_name in no_prior:
    
    print(gw_name)
    
    with h5py.File(f'IGWN-GWTC2p1-v2-{gw_name}_PEDataRelease_mixed_cosmo.h5') as f:
        df = pd.DataFrame(np.asarray(f['C01:Mixed']['posterior_samples'])) 

    # luminosity distance and redshift
    dL = np.asarray(df['luminosity_distance'])
    z = np.asarray(df['redshift'])

    # chirp mass and total mass (source frame)
    m1_src = np.asarray(df['mass_1_source'])
    m2_src = np.asarray(df['mass_2_source'])
    mchirp = np.asarray(df['chirp_mass_source'])
    mtot = np.asarray(df['total_mass_source'])

    # q and eta
    q = np.asarray(df['mass_ratio'])
    eta = np.asarray(df['symmetric_mass_ratio'])

    # chieff
    chieff = components_to_chieff(m1_src, m2_src, np.asarray(df['a_1']), np.asarray(df['a_2']), \
                                  np.asarray(df['cos_tilt_1']), np.asarray(df['cos_tilt_2']))

    new_df = pd.DataFrame({'m1':m1_src, 'm2':m2_src, 'mchirp':mchirp, 'mtot':mtot, 'q':q, 'eta':eta, 
                           'chieff':chieff, 'dL':dL, 'z':z})  
    
    bb_priors = bilby.core.prior.PriorDict(filename=f'{output_directory}/../extra_event_priors/{gw_name}_prior.prior')
    
    z_min = dL_to_redshift(bb_priors['luminosity_distance'].minimum)
    z_max = dL_to_redshift(bb_priors['luminosity_distance'].maximum)
    a_max = bb_priors['a_1'].maximum

    bb_priors.pop('luminosity_distance')
    bb_priors['redshift'] = bilby.gw.prior.UniformSourceFrame(z_min, z_max, name='redshift')

    priors = bilby.core.prior.PriorDict()
    for param in prior_params_z:
        priors[param] = bb_priors[param]

    p = prob(priors, df[prior_params_z]) #mchirp_det, q, z

    p *= (1 + new_df['z']) #mchirp_det to mchirp_source jacobian

    #get chieff prior from tcallister

    p_chieffs = []
    for q_i, chieff_i in zip(new_df['q'], new_df['chieff']):
        p_chieffs.append(chi_effective_prior_from_isotropic_spins(q_i, a_max, chieff_i)[0]) #p(chieff | q)

    p *= np.array(p_chieffs)

    new_df['p_theta_jcb'] = p
    
    new_df.to_hdf(os.path.join(output_directory,gw_name+'.hdf5'), key='combined')

GW170608_020116


/tmp/ipykernel_782026/3106650293.py:44: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  p = prob(priors, df[prior_params_z]) #mchirp_det, q, z


GW190707_093326


/tmp/ipykernel_782026/3106650293.py:44: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  p = prob(priors, df[prior_params_z]) #mchirp_det, q, z


GW190720_000836


...

/tmp/ipykernel_782026/3106650293.py:44: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  p = prob(priors, df[prior_params_z]) #mchirp_det, q, z


GW190728_064510


/tmp/ipykernel_782026/3106650293.py:44: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  p = prob(priors, df[prior_params_z]) #mchirp_det, q, z


GW190924_021846


/tmp/ipykernel_782026/3106650293.py:44: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  p = prob(priors, df[prior_params_z]) #mchirp_det, q, z
